# 데이터 불러오기

In [56]:
# from google.colab import drive
# drive.mount('/content/drive')

- 시드값

In [1]:
SEED = 42

- 데이터 불러오기

In [17]:
import pandas as pd
import numpy as np

DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/"
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [29]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.2_1107.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.2_1107.csv")

train_ft.shape , test_ft.shape

((14940, 1455), (12225, 1455))

# 결측치 처리

In [30]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [31]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1454), (12225, 1454))

- 추가 피처 만들어 보기

In [62]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

## Feature Encoding

In [32]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [33]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1458), (12225, 1458))

In [34]:
import category_encoders as ce

enc = ce.count.CountEncoder()
# train_ft["주구매지점_cnt"] = enc.fit_transform(train_ft[["주구매지점"]])
# test_ft["주구매지점_cnt"] = enc.transform(test_ft[["주구매지점"]])

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])
train_ft.shape, test_ft.shape

((14940, 1460), (12225, 1460))

- 문자열 피처 삭제

In [35]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [36]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1457), (12225, 1457))

In [37]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

- 가중치 적용

In [69]:
train_ft["18시_21시_구매비율"] = train_ft["18시_21시_구매비율"] * 3
train_ft["18시_21시_구매횟수"] = train_ft["18시_21시_구매횟수"] * 3

test_ft["18시_21시_구매비율"] = test_ft["18시_21시_구매비율"] * 3
test_ft["18시_21시_구매횟수"] = test_ft["18시_21시_구매횟수"] * 3

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,...,-0.162079,-0.286379,-0.209907,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,...,2.389509,-0.286379,-0.209907,4.161199,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.286379,-0.209907,-0.213001,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,...,10.044273,-0.286379,1.301424,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.452496,-0.613193,0.302875,-0.302875,0.838830,0.567300,-0.752532,1.707410,-0.130285,-0.821561,...,-0.162079,4.377687,4.324087,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


# 정답 데이터

In [27]:
target = train_target["target"]

# cv 점수 확인해보기

### xgboost

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


# v3.0 원핫 + 스탠다드 : 0.7195019304392802
# submit_v3.0_02 : 원핫(주구매지점) + 카운트(주구매대분류, 주구매중분류) + 스탠다드  (0.7202916471187197)
# submit_v3.0_02 : 원핫(주구매지점) + 카운트(주구매대분류, 주구매중분류) + 스탠다드 + 가중치적용  ( 0.7202916471187197 )
# 3.2 셋다 카운트 + 스탠다드 (0.7184464536223091)

Mean F1 Macro Score: 0.7184464536223091


In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

# from lightgbm import LGBMClassifier

# model = LGBMClassifier(random_state=SEED)
# scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
# np.mean(scores)

In [ ]:
model = XGBClassifier(**params)
model.fit(train_ft,target)
pred = model.predict(test_ft)
submit["target"] = pred

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6290102054237857, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.648553153047272,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04036413044768581,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=800, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

- 예측 결과를 csv 파일로 저장하여 제출

In [49]:
DATA_PATH = 'C:/Users/user/Desktop/데이터분석/05 Project_Final/Submit_csv/'
submit.to_csv(f"{DATA_PATH}submit_v3.0_1106.csv",index=False)

## automl

In [ ]:
from flaml import AutoML
import xgboost 
from xgboost import XGBClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*70,
           "seed" : 42,
           "early_stop" : True,
           "eval_method": "cv",                 # 교차 검증
           "n_splits": 10,                      # Stratified K-Fold로 10분할 교차 검증
           "ensemble" : {'final_estimator' : XGBClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2']  }   # 앙상블에 사용할 모델 지정

auto_ml_ens.fit(train_ft, target, **params)

# 

## 피처셀렉션

In [ ]:
from tqdm.notebook import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = []
for percentile in tqdm(range(1,101)):
    sp = SelectPercentile(percentile=percentile)
    x = sp.fit_transform(train_ft, target)
    score = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    scores.append([percentile, score])
scores = np.array(scores)
idx = np.argmax(scores[:,1])
best_score = scores[idx]

sp = SelectPercentile(percentile=best_score[0])
sp.fit(train_ft,target)
best_cols = sp.get_feature_names_out()
len(best_cols)

  0%|          | 0/100 [00:00<?, ?it/s]

array(['내점일수', '구매주기', '주말방문비율', '평일방문비율', '주말방문횟수', '평일방문횟수', '봄_구매비율',
       '여름_구매비율', '겨울_구매비율', '주구매요일', '주구매_년', '주구매_월', '주구매_시간대',
       '일별평균구매횟수', '거래개월수', '9시_12시_구매비율', '12시_15시_구매비율', '18시_21시_구매비율',
       '9시_12시_구매횟수', '12시_15시_구매횟수', '18시_21시_구매횟수', '월초_구매횟수',
       '월말_구매횟수', '웨딩성수기_구매횟수', '1월_구매비율', '2월_구매비율', '3월_구매비율',
       '4월_구매비율', '5월_구매비율', '6월_구매비율', '7월_구매비율', '8월_구매비율', '10월_구매비율',
       '12월_구매비율', '1월_구매횟수', '2월_구매횟수', '3월_구매횟수', '4월_구매횟수', '5월_구매횟수',
       '6월_구매횟수', '7월_구매횟수', '8월_구매횟수', '9월_구매횟수', '10월_구매횟수', '11월_구매횟수',
       '12월_구매횟수', '1월_방문횟수', '2월_방문횟수', '3월_방문횟수', '4월_방문횟수', '5월_방문횟수',
       '6월_방문횟수', '7월_방문횟수', '8월_방문횟수', '9월_방문횟수', '10월_방문횟수', '11월_방문횟수',
       '12월_방문횟수', '공휴일_구매유무', '여름휴가_구매유무', '연말_구매유무', '여름휴가_구매비율',
       '연말_구매비율', '공휴일_구매횟수', '여름휴가_구매횟수', '연말_구매횟수', '1월_총구매금액',
       '2월_총구매금액', '3월_총구매금액', '4월_총구매금액', '5월_총구매금액', '6월_총구매금액',
       '7월_총구매금액', '8월_총구매금액', '9월_총구매금액', '10월_총구매금액', '11월_총구매금액',
       '12월_총

In [43]:
tmp = train_ft[best_cols]

In [44]:
from flaml import AutoML
import xgboost 
from xgboost import XGBClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*20,
           "seed" : 42,
           "early_stop" : True,
           "eval_method": "cv",                 # 교차 검증
           "n_splits": 10,                      # Stratified K-Fold로 10분할 교차 검증
           "ensemble" : {'final_estimator' : XGBClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2']  }   # 앙상블에 사용할 모델 지정

auto_ml_ens.fit(tmp, target, **params)

# 

[flaml.automl.logger: 11-07 15:26:03] {1728} INFO - task = classification
[flaml.automl.logger: 11-07 15:26:03] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-07 15:26:03] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-07 15:26:03] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2']
[flaml.automl.logger: 11-07 15:26:03] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-07 15:26:52] {2393} INFO - Estimated sufficient time budget=487344s. Estimated necessary time budget=668s.
[flaml.automl.logger: 11-07 15:26:52] {2442} INFO -  at 50.4s,	estimator catboost's best error=0.2944,	best estimator catboost's best error=0.2944
[flaml.automl.logger: 11-07 15:26:52] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-07 15:27:00] {2442} INFO -  at 58.7s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.2944
[flaml.automl.l

In [61]:
1-sum(auto_ml_ens.best_loss_per_estimator.values())/len(auto_ml_ens.best_loss_per_estimator)

0.7079270014995

In [48]:
from flaml import AutoML
import xgboost 
from xgboost import XGBClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*20,
           "seed" : 42,
           "early_stop" : True,
           "eval_method": "cv",                 # 교차 검증
           "n_splits": 10,                      # Stratified K-Fold로 10분할 교차 검증
           "ensemble" : True,    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb']  }   # 앙상블에 사용할 모델 지정

auto_ml_ens.fit(tmp, target, **params)

# 

[flaml.automl.logger: 11-07 15:50:16] {1728} INFO - task = classification
[flaml.automl.logger: 11-07 15:50:16] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-07 15:50:16] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-07 15:50:16] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb']
[flaml.automl.logger: 11-07 15:50:16] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-07 15:51:05] {2393} INFO - Estimated sufficient time budget=484938s. Estimated necessary time budget=485s.
[flaml.automl.logger: 11-07 15:51:05] {2442} INFO -  at 50.1s,	estimator catboost's best error=0.2944,	best estimator catboost's best error=0.2944
[flaml.automl.logger: 11-07 15:51:05] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-07 15:51:13] {2442} INFO -  at 58.5s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.2944
[flaml.automl.logger: 11-07 15:

In [49]:
auto_ml_ens.best_loss_per_estimator

{'catboost': 0.2927925734561826,
 'lgbm': 0.29007837477623877,
 'rf': 0.298499804898915,
 'xgboost': 0.28564726103378735,
 'histgb': 0.2933469783373758}

In [62]:
1-sum(auto_ml_ens.best_loss_per_estimator.values())/len(auto_ml_ens.best_loss_per_estimator)

0.7079270014995

In [69]:
auto_ml_ens.best_config_per_estimator.get("catboost")

{'early_stopping_rounds': 11,
 'learning_rate': 0.060536189750294574,
 'n_estimators': 113}

In [71]:
best_1 = auto_ml_ens.best_config_per_estimator.get("xgboost")
best_2 = auto_ml_ens.best_config_per_estimator.get("catboost")
best_3 = auto_ml_ens.best_config_per_estimator.get("lgbm")

from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier(
    estimators= [
        ("best_1", best_1),
        ("best_2", best_2),
        ("best_3", best_3),
    ], voting= "soft"
)

ensemble_model.fit(x_train, y_train)
f1_score(x_valid, y_valid, average="macro")

ValueError: The estimator dict should be a classifier.

In [63]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(tmp, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


# v3.0 원핫 + 스탠다드 : 0.7195019304392802
# submit_v3.0_02 : 원핫(주구매지점) + 카운트(주구매대분류, 주구매중분류) + 스탠다드  (0.7202916471187197)
# submit_v3.0_02 : 원핫(주구매지점) + 카운트(주구매대분류, 주구매중분류) + 스탠다드 + 가중치적용  ( 0.7202916471187197 )
# 3.2 셋다 카운트 + 스탠다드 (0.7184464536223091)

Mean F1 Macro Score: 0.7191309247065132
